In [183]:
import spacy #nlp
import re #regex
import os #file manip
import pandas as pd #csv - tabular
import sqlite3
import pdf2txt
import shutil
from tkinter import *
from tkinter import filedialog

In [184]:
count=0 #if count 0 then don't create txt folder and csv folder + file
#if no files in pdf (i.e. count==0), print no files in pdf

In [185]:

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.

root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
file_dir = filedialog.askdirectory()
txtpath = file_dir + '/output/txt/' #txtpath will be deleted later

In [186]:
# connecting to db
conn = sqlite3.connect('persisting_database')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS candidates (candidate_name text, phone number, candidate_email text, candidate_skill text)')
conn.commit()

In [187]:
def convert_pdf(f):

    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt" #Split filename and add .pdf

    newpath = file_dir + '/output/txt/' #same statement as above, but txtpath would be deleted
    if not os.path.exists(newpath):
        os.makedirs(newpath)    


    output_filepath = os.path.join(newpath, output_filename) #location for saving txt files
    pdf2txt.main(args=[f, "--outfile", output_filepath]) #pdf to txt and save it in the given location
    print(output_filepath + " saved successfully!!!")  #print the message
    return open(output_filepath, encoding="utf8").read() #read the file, encoding utf8 cuz giving error 
 #                                              #without specifying encoding while converting some pdf files 

In [188]:
nlp = spacy.load("en_core_web_sm")  # load the language model

In [189]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []}  #creaing the dictionary
names = []  #list for storing respective values
phones = []
emails = []
skills = []

In [190]:
def parse_content(text):  #takes 1 arg
    skillset = re.compile("python|java|sql|aws|c|linux|mysql") #skills we want to extract, you can type anything
    phone_num = re.compile(
        "(\d{12}|\d{7}[-]??\d{5}|\d{13}|\d{10}|\d{5}[-]??\d{5})"  #indian phone numbers' possible combinations
    ) #number we want to extract
    doc = nlp(text) #annotate the text using nlp
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]  #extracting 1st name
    print(name)
    email = [word for word in doc if word.like_email == True][0]   #extracting 1st email
    print(email)
    phone = str(re.findall(phone_num, text.lower()))  #convert text to lower case and find all
    skills_list = re.findall(skillset, text.lower())  
    unique_skills_list = str(set(skills_list))  #we want unique skills list, convert to set
    names.append(name)  #place the values in the above lists
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Zaid\AppData\Local\Temp/ipykernel_17212/2419067926.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]  #extracting 1st name


In [191]:
for file in os.listdir(file_dir):  #list all files and go through them all
    if file.endswith('.pdf'):  #only works with pdf so check that
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join(file_dir, file)) #calling the function #assigning convert_pdf output to txt
        parse_content(txt) #then pass the txt to parse_content
        count+=1 #for every pdf converted, count+1
    elif(count==0): 
        print('Files are not in pdf format')
        continue

Reading.....AshleyMilesCV.pdf
D:/Projects/Sem 7/Mini Project/Code/Semi-Auto/resumes/output/txt/AshleyMilesCV.txt saved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!
Reading.....Devendra S_Btech CTIS_Resume.pdf
D:/Projects/Sem 7/Mini Project/Code/Semi-Auto/resumes/output/txt/Devendra S_Btech CTIS_Resume.txt saved successfully!!!
Devendra Sunil Sawande
Devendra.sawande@adypu.edu.in
Extraction completed successfully!!!
Reading.....Digvijay Zanjurne Resume.pdf
D:/Projects/Sem 7/Mini Project/Code/Semi-Auto/resumes/output/txt/Digvijay Zanjurne Resume.txt saved successfully!!!
Digvijay Sunil Zanjurne
digvijayzanjurne2001@gmail.com
Extraction completed successfully!!!
Reading.....Harshada Wadekar_ Btech_CTIS_Resume.pdf
D:/Projects/Sem 7/Mini Project/Code/Semi-Auto/resumes/output/txt/Harshada Wadekar_ Btech_CTIS_Resume.txt saved successfully!!!
Harshada Subhash Wadekar
harshada.wadekar@adypu.edu.in
Extraction completed successfully!!!
Reading.....John 

In [192]:
result_dict['name'] = names #assigning the values of list to dict
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills

In [193]:
result_df = pd.DataFrame(result_dict) #convert into tabular format using df 

#DELETE TXT FOLDER AFTER PROGRAM IS FINISHED
try:    
    shutil.rmtree(txtpath)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))

In [194]:
if count>0:
    newcsvpath = file_dir + '/output/csv/' 
    if not os.path.exists(newcsvpath):
        os.makedirs(newcsvpath)   
    csvname = newcsvpath + 'parsed_resumes.csv'
    result_df.to_csv(csvname)
    os.startfile(csvname)

    

In [195]:
#converting datatype
result_df['email']= result_df['email'].astype('str')

#feeding values to database
result_df.to_sql('candidates', conn, if_exists='append', index = False)

#deleting duplicates
c.execute('''DELETE FROM candidates 
   WHERE  rowid NOT IN ( 
      SELECT min(rowid) 
      FROM   candidates 
      GROUP  BY name, phone);''') 
   
conn.commit()

#result_df.to_sql('candidates', conn, if_exists='append', index = False)

result_dfdb = pd.DataFrame(c.execute('''  
   SELECT * FROM candidates
             '''))     
result_dfdb

#Made by Zaid Qureshi with Coffee & Love

,0,1,2,3
0,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'c', 'sql'}"
1,Devendra Sunil Sawande,"['9067918225', '9975199997']",Devendra.sawande@adypu.edu.in,"{'java', 'c'}"
2,Digvijay Sunil Zanjurne,"['9130840095', '9970940666']",digvijayzanjurne2001@gmail.com,"{'java', 'python', 'linux', 'c'}"
3,Harshada Subhash Wadekar,"['9168379021', '9168379021']",harshada.wadekar@adypu.edu.in,"{'java', 'c'}"
4,John Dominic,['7877756411'],johndominic@mail.com,"{'java', 'python', 'c'}"
